In [1]:
import torch
import torch.nn.functional as F
import numpy as np

from ssgd import StreamingSGD

# Model definition

A StreamingSGD compatible model now needs to be able to "detach" layers as well as gather input/output and gradients. It also needs a list of layers. See below for implementation example. In the future we want to implement this using PyTorch hooks and modules.

In [2]:
class ExampleNet(torch.nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        
        self.input_layer = torch.nn.Conv2d(3, 3, kernel_size=3)
        self.layers = [self.input_layer]

        for i in range(5):  # use 9 for 8194 x 8194 images
            self.add_block(i)
        
        final_conv5 = torch.nn.Conv2d(3, 1, kernel_size=10)
        self.add_module("final", final_conv5)
        
        self.layers.extend([final_conv5])
        
    def add_block(self, i):
        conv1 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv2 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv3 = torch.nn.Conv2d(3, 3, kernel_size=3)
        maxpool = torch.nn.MaxPool2d(2, stride=2)
        
        self.add_module("conv1-" + str(i), conv1)
        self.add_module("conv2-" + str(i), conv2)
        self.add_module("conv3-" + str(i), conv3)
        self.add_module("maxpool-" + str(i), maxpool)

        self.layers.extend([conv1, conv2, conv3, maxpool])
        
    def backward(self, gradient):
        self.gradients = []
        for i, output in reversed(list(enumerate(self.output))):
            if i < (len(self.output) - 1):
                gradient = self.input[i+1].grad
            output.backward(gradient=gradient, retain_graph=True)
            self.gradients.append(gradient)

    def forward(self, x, stop_index=-1, start_index=0, detach=False):
        if detach:
            self.output = []
            self.input = []
        for i, layer in enumerate(self.layers[start_index:]):
            if detach:
                x = torch.autograd.Variable(x.data, requires_grad=detach)
                self.input.append(x)
            if i == stop_index:
                break
            if i + 1 == len(self.layers[start_index:]):
                x = layer(x)
                x = x.view(-1, 1)
                x = F.sigmoid(x)
            else:
                x = F.relu(layer(x))
            if detach:
                self.output.append(x)
        return x

model = ExampleNet()
# model = model.double()

Weight initialization; we use positive values to generate large gradients, better for testing if final gradients are correct.

In [3]:
for i, layer in enumerate(model.modules()):
    if isinstance(layer, torch.nn.Conv2d):
        layer.weight.data.fill_(0.04)
        layer.bias.data.zero_()

In [4]:
print(model)

ExampleNet(
  (input_layer): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv1-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-0): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-3): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-3): Conv2d(3, 3, kerne

In [5]:
sCNN = StreamingSGD(model)

# Configurations

In [6]:
stop_index = 14  # use 21 for 8194x8194 images
img_size = 514  # try 8194, see last segment for details

cuda = False  # execute this notebook on the GPU
verbose = True  # enable / disable logging
divide_in = 8  # tip: use 25 for 8194x8194 when memory constraint

# Configure streaming SGD

In [7]:
if cuda:
    model.cuda()

In [8]:
sCNN.configure(model.layers, stop_index, (img_size, img_size, 3), divide_in, cuda, verbose)

Calculating patch boxes...
[1, 3, 512.0, 512.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 510.0, 510.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 508.0, 508.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 506.0, 506.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 253.0, 253.0] [2.0, 2.0] [0.0, 0.0, 0.0, 0.0] MaxPool2d
[1, 3, 251.0, 251.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 249.0, 249.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 247.0, 247.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 123.0, 123.0] [2.0, 2.0] [0.0, 1.0, 0.0, 1.0] MaxPool2d
[1, 3, 121.0, 121.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 119.0, 119.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 117.0, 117.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 58.0, 58.0] [2.0, 2.0] [0.0, 1.0, 0.0, 1.0] MaxPool2d
[1, 3, 56.0, 56.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
Embedding divided in tile sizes: (7, 7) 

[1, 3, 184.0, 184.0] [1.0, 1.0] [1.0, 1.0, 1.0, 1.0] Conv2d
[1, 3, 182.0, 182.0] [1.0,

# Generate random image and fake label

In the current implementation the whole image needs to be able to fit in memory (RAM).

In [9]:
image = torch.FloatTensor(3, img_size, img_size).normal_(0, 1)
target = torch.FloatTensor(1, 1).fill_(0)

# image = image.double()
# target = target.double()

if cuda:
    target = target.cuda()

In [10]:
image_var = torch.autograd.Variable(image)
# image_var = image_var.double()

In [11]:
criterion = torch.nn.BCELoss()

In [12]:
output, feature_map = sCNN.forward(image_var)

  0%|          | 0/64 [00:00<?, ?it/s]

Doing forward pass...


100%|██████████| 64/64 [00:00<00:00, 428.28it/s]


In [13]:
output

Variable containing:
 0.9895
[torch.FloatTensor of size 1x1]

In [14]:
loss = criterion(output, torch.autograd.Variable(target)); loss

Variable containing:
 4.5609
[torch.FloatTensor of size 1]

In [15]:
full_gradients = sCNN.backward(image_var, feature_map, loss)

  0%|          | 0/64 [00:00<?, ?it/s]

Doing backward pass...


100%|██████████| 64/64 [00:01<00:00, 38.49it/s]


Filled gradient sizes:

   53    56    56
   53    58    58
  105   117   117
  105   119   119
  105   121   121
  105   123   123
  209   247   247
  209   249   249
  209   251   251
  209   253   253
  418   506   506
  418   508   508
  418   510   510
  418   512   512
[torch.FloatTensor of size 14x3]
 

Everything filled:
 True


"Everything filled" means that all gradients were reconstructed succesfully!

Save the gradients of the conv2d layer to compare with normal SGD:

In [16]:
streaming_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            streaming_conv_gradients.append(layer.weight.grad.clone()) 

# Compare to normal SGD

Reset the gradients and perform a normal for backward pass.

In [17]:
for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            layer.weight.grad.data.zero_()
            layer.bias.grad.data.zero_()

In [18]:
output = model.forward(image_var[None]);

In [19]:
output

Variable containing:
 0.9895
[torch.FloatTensor of size 1x1]

This output should be the same as the streaming SGD output, if so the loss will also be the same:

In [20]:
loss = criterion(output, torch.autograd.Variable(target)); loss

Variable containing:
 4.5609
[torch.FloatTensor of size 1]

Here we do a normal PyTorch backward pass:

In [21]:
loss.backward()

# Compare the gradients of the conv2d layers

In [22]:
normal_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            normal_conv_gradients.append(layer.weight.grad) 

In [23]:
for i in range(len(streaming_conv_gradients)):
    print("Layer", i, "\taverage gradient size:", 
          torch.mean(streaming_conv_gradients[i].data))

Layer 0 	average gradient size: 1.3897795191517583
Layer 1 	average gradient size: 1.3897795677185059
Layer 2 	average gradient size: 1.3897795677185059
Layer 3 	average gradient size: 1.3897795147365994
Layer 4 	average gradient size: 1.3897796030397769
Layer 5 	average gradient size: 1.3897795323972348
Layer 6 	average gradient size: 1.3897795059062816
Layer 7 	average gradient size: 1.389779598624618
Layer 8 	average gradient size: 1.3897795809639826
Layer 9 	average gradient size: 1.3897795412275527
Layer 10 	average gradient size: 1.3897795412275527
Layer 11 	average gradient size: 1.3897795412275527
Layer 12 	average gradient size: 1.3897795014911227
Layer 13 	average gradient size: 1.3897795014911227
Layer 14 	average gradient size: 1.3897794485092163
Layer 15 	average gradient size: 1.389779461754693
Layer 16 	average gradient size: 0.3752404388785362


In [24]:
for i in range(len(streaming_conv_gradients)):
    max_diff = torch.max(torch.abs(streaming_conv_gradients[i].data - 
                      normal_conv_gradients[i].data))
    print("Layer", i, "\tmax difference between gradients:", max_diff)

Layer 0 	max difference between gradients: 1.7881393432617188e-06
Layer 1 	max difference between gradients: 1.6689300537109375e-06
Layer 2 	max difference between gradients: 1.3113021850585938e-06
Layer 3 	max difference between gradients: 2.86102294921875e-06
Layer 4 	max difference between gradients: 8.344650268554688e-07
Layer 5 	max difference between gradients: 7.152557373046875e-07
Layer 6 	max difference between gradients: 7.152557373046875e-07
Layer 7 	max difference between gradients: 1.0728836059570312e-06
Layer 8 	max difference between gradients: 5.960464477539062e-07
Layer 9 	max difference between gradients: 3.5762786865234375e-07
Layer 10 	max difference between gradients: 4.76837158203125e-07
Layer 11 	max difference between gradients: 0.0
Layer 12 	max difference between gradients: 0.0
Layer 13 	max difference between gradients: 0.0
Layer 14 	max difference between gradients: 0.0
Layer 15 	max difference between gradients: 0.0
Layer 16 	max difference between gradient

As you can see the difference of the gradients of the conv2d layers between the methods is (almost) numerically equivalent. The small differences are because of loss of significance with the floating points calculations. 

---

# Things to try:

* Use doubles instead of floats to reduce the difference (use model.double() and image_var.double())
* Make the image bigger than would fit on a GPU 
    - e.g. 8194x8194, make sure to add 3 more blocks in the model (see comments)
* If you want you can compare the reconstructed input gradients of each layer: 
    - pass fill_gradient=True in backward() function
    - compare full_gradients with self.model.gradients after the full model backward pass.
* For testing purposes the number of filters is small in this notebook, try increasing them